# SAD2- Project 1
Maria Nizik

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px

## 1 
Joint probabilities:  

$P(E=T, F=T, J=T, M=T) = P(E=T) \cdot P(F=T|E=T) \cdot (J=T|E=T) \cdot P(M=T|F=T,J=T) = 0.5 \cdot 0.1 \cdot 0.8 \cdot 0.95 = 0.038$  

$P(E=F, F=T, J=T, M=T) = (1 - P(E=T)) \cdot P(F=T|E=F) \cdot (J=T|E=F) \cdot P(M=T|F=T,J=T) = 0.5 \cdot 0.5 \cdot 0.1 \cdot 0.95 = 0.02375$  

$P(E=T, F=F, J=T, M=T) = P(E=T) \cdot (1 - P(F=T|E=T)) \cdot (J=T|E=T) \cdot P(M=T|F=F,J=T) = 0.5 \cdot 0.9 \cdot 0.8 \cdot 0.9 = 0.324$  

$P(E=F, F=F, J=T, M=T) = (1 - P(E=F)) \cdot (1 - P(F=T|E=F)) \cdot (J=T|E=F) \cdot P(M=T|F=F,J=T) = 0.5 \cdot 0.5 \cdot 0.1 \cdot 0.9 = 0.0225$

Conditional probabilities:  

$P(E=T | F=T, J=T, M=T) = \frac{P(E=T, F=T, J=T, M=T)}{\sum_E P(E, F=T, J=T, M=T)} = \frac{P(E=T, F=T, J=T, M=T)}{P(E=T, F=T, J=T, M=T) + P(E=F, F=T, J=T, M=T)} = \frac{0.038}{0.038 + 0.02375} \approx 0.615$ 

$P(E=T | F=F, J=T, M=T) = \frac{P(E=T, F=F, J=T, M=T)}{\sum_E P(E, F=F, J=T, M=T)} = \frac{P(E=T, F=F, J=T, M=T)}{P(E=T, F=F, J=T, M=T)+P(E=F, F=F, J=T, M=T)} = \frac{0.324}{0.324+0.0225} \approx 0.935$  

$P(F=T | E=T, J=T, M=T) = \frac{P(E=T, F=T, J=T, M=T)}{\sum_F P(F, E=T, J=T, M=T)} = \frac{P(E=T, F=T, J=T, M=T)}{P(E=T, F=T, J=T, M=T) + P(E=T, F=F, J=T, M=T)} = \frac{0.038}{0.038 + 0.324} \approx 0.105$  

$P(F=T | E=F, J=T, M=T) = \frac{P(E=F, F=T, J=T, M=T)}{\sum_F P(F, E=F, J=T, M=T)} = \frac{P(E=F, F=T, J=T, M=T)}{P(E=F, F=T, J=T, M=T)+P(E=F, F=F, J=T, M=T)} = \frac{0.02375}{0.02375+0.0225} \approx 0.514$  




## 2

In [3]:
class Model:
    """Mortgage model"""

    def __init__(self):
        """P(X=F) are included. 
        Example: [[P(False|False), P(False|True)], [P(True|False), P(True|True)]]"""
        self.p_e = np.array([0.5, 0.5])

        self.p_f_on_e = np.array([[0.5, 0.9],
                               [0.5, 0.1]])
        
        self.p_j_on_e = np.array([[0.9, 0.2],
                               [0.1, 0.8]])
        
        self.p_m_on_fj = np.array([[[0.9, 0.1],
                                 [0.1, 0.05]],
                                 [[0.1, 0.9],
                                 [0.9, 0.95]]])

    def count_conditional(self, variable: list, states: np.array) -> tuple[np.array]:
        e, f, j, m = states

        nominator = self.p_e[e] * self.p_f_on_e[f][e] * self.p_j_on_e[j][e] * self.p_m_on_fj[m][f][j]

        states[variable[0]] ^= 1
        e, f, _, _ = states

        denominator = nominator + self.p_e[e] * self.p_f_on_e[f][e] * self.p_j_on_e[j][e] * self.p_m_on_fj[m][f][j]
        
        return nominator / denominator
    
model = Model()

In [4]:
# Model tests
first = model.count_conditional([0, 'e'], np.array([1,1,1,1]))
second = model.count_conditional([0, 'e'], np.array([1,0,1,1]))
third = model.count_conditional([1, 'f'], np.array([1,1,1,1]))
fourth = model.count_conditional([1, 'f'], np.array([0,1,1,1]))
print(first, second, third, fourth)

fi = model.count_conditional([0, 'e'], np.array([0,1,1,1]))
s = model.count_conditional([0, 'e'], np.array([0,0,1,1]))
t = model.count_conditional([1, 'f'], np.array([1,0,1,1]))
fo = model.count_conditional([1, 'f'], np.array([0,0,1,1]))
print(fi, s, t, fo)

0.6153846153846154 0.935064935064935 0.10497237569060772 0.5135135135135135
0.3846153846153846 0.06493506493506493 0.8950276243093922 0.48648648648648657


In [5]:
class Sampler:
    """Gibbs sampler"""
    def __init__(self, model: Model, environment: int = 1, funding: int = 1, job: int = 1, mortgage: int = 1):
        self.model = model
        self.e = environment
        self.f = funding
        self.j = job
        self.m =  mortgage

    def states(self) -> np.array:
        return np.array([self.e, self.f, self.j, self.m])  
            
    def sample(self, sampling_iters: int = 1, burn_in_iters:int = 0, thin_by: int = 1) -> np.array:
        """Thin_by takes thining-out parameter. For thin = 1 there is no thinning and all samples are given."""

        iters = burn_in_iters + thin_by * sampling_iters
        samples = []

        for _ in range(iters):
            i = np.random.choice([0,1])
            variable = ([1, 'f'] if i else [0, 'e'])
            prob = self.model.count_conditional(variable, self.states())

            current_state = getattr(self, variable[1])
            opposite_state = current_state ^ 1
            new_state = np.random.choice([current_state, opposite_state], p = [prob, 1 - prob])

            setattr(self, variable[1], new_state)
            samples.append(self.states())

        return np.array(samples[burn_in_iters::thin_by])
    

np.random.seed(123)
sampler = Sampler(model) 
samples = sampler.sample(100)

## 3

In [6]:
print('P(F = T|J = T, M = T) =', np.mean(samples[:, 1]))

P(F = T|J = T, M = T) = 0.12


## 4

In [7]:
np.random.seed(100)
sampler2 = Sampler(model) 
samples2 = sampler2.sample(50000)

## 5

In [8]:
# Third run of a sampler
np.random.seed(51)
samler3 = Sampler(model, environment=0, funding=0)
samples3 = sampler.sample(50000)

In [9]:
def count_frequencies(sample_arrays: np.ndarray, cols_of_interest: list[int]) -> np.ndarray:
    """Computes relative frequencies and converts arrays of samples to easy-to-plot dataframes"""
    
    assert isinstance(sample_arrays, np.ndarray)

    sample_arrays = sample_arrays[ : , : , cols_of_interest]
    index_array = np.arange(1, len(sample_arrays[0]) + 1)[ : , np.newaxis]
    means = np.cumsum(sample_arrays, axis = 1) / index_array
    
    return means


def stats_to_df(data: np.ndarray, stat_name: str, sampler_nr: int) -> pd.DataFrame:
    """Converts stats array to easy-to-plot dataframe."""

    assert isinstance(data, np.ndarray)
    assert data.ndim == 2

    df = pd.DataFrame(data, columns=['E', 'F'])
    df['iteration'] = df.index
    df = df.melt(id_vars=['iteration'], value_name=stat_name)
    df['sampler'] = sampler_nr

    return df


def stats_to_combined_df(stats_arrays: np.ndarray, stat_name: str, sampler_nrs: list) -> pd.DataFrame:

    assert isinstance(stats_arrays, np.ndarray)
    assert stats_arrays.ndim == 3

    dfs = []

    for array, nr in zip(stats_arrays, sampler_nrs):
        df = stats_to_df(array, stat_name, nr)
        dfs.append(df)

    combined = pd.concat(dfs, ignore_index = True) 

    return combined


frequencies = count_frequencies(np.array([samples2, samples3]), [0, 1])
combined_frequencies_df = stats_to_combined_df(frequencies, 'frequency', [2, 3])
display(combined_frequencies_df)

,iteration,variable,frequency,sampler
0,0,E,0.000000,2
1,1,E,0.000000,2
2,2,E,0.000000,2
3,3,E,0.000000,2
4,4,E,0.000000,2
...,...,...,...,...
199995,49995,F,0.148232,3
199996,49996,F,0.148229,3
199997,49997,F,0.148226,3
199998,49998,F,0.148223,3


In [10]:
def frequencies_plot(df: pd.DataFrame):

    assert isinstance(df, pd.DataFrame)

    plt = px.line(df,
                x = 'iteration',
                y = 'frequency',
                facet_col = 'variable',
                color = 'sampler',
                title =  'E and F relative frequncies as Gibbs samplers progress',
                height = 500,
                width = 1000)
    
    plt.write_image('frequencies.png', scale = 8)


frequencies_plot(combined_frequencies_df)

![Plot](./frequencies.png)

E = T relative frequencies converge after ~40k iterations and F = T after ~34k iterations. I would use the higher of these values (40k) as a burn-in time.

## 6  
The next 2 sets of 50000 samples are prepared. Each is initialised separately and with different seed to ensure independence, while mantaining reproducibility. Seeds were chosen once and arbitrarily. Each of samplers was initialised with different E and F values and no burn-in time: sampler2 (E = T, F = T), sampler3 (E = F, F = F), sampler4 (E = T, F = F), and sampler5 (E = F, F = T).

In [11]:
np.random.seed(133)
sampler4 = Sampler(model, environment=1, funding=0)
samples4 = sampler4.sample(50000)

In [12]:
np.random.seed(17)
sampler5 = Sampler(model, environment=0, funding=1)
samples5 = sampler5.sample(50000)

In [13]:
def gelman_test(sample_arrays: np.ndarray, cols_of_interest: list[int]) -> np.ndarray:
    """Gelman-Rubin statistic."""

    assert isinstance(sample_arrays, np.ndarray)
    assert sample_arrays.shape[0] > 1

    sample_arrays = sample_arrays[ : , : , cols_of_interest]
    index_array = np.arange(1, len(sample_arrays[0]) + 1)[ : , np.newaxis]

    means = np.cumsum(sample_arrays, axis = 1) / index_array
    grand_means = np.mean(means, axis = 0)
    between_vars = np.sum((means - grand_means) ** 2, axis = 0) / (len(sample_arrays) - 1)
    within_vars = np.sum((sample_arrays - means) ** 2, axis = 0) / index_array
    ws= np.mean(within_vars, axis = 0) / len(sample_arrays)

    result = (ws + between_vars) / ws

    return result

gelman_samples = np.array([samples2, samples3, samples4, samples5])
gelman_result  = gelman_test(gelman_samples, [0,1])

In [14]:
gelman_df = stats_to_df(gelman_result, 'gelman', 2)
display(gelman_df)

,iteration,variable,gelman,sampler
0,0,E,15575.076235,2
1,1,E,15575.076235,2
2,2,E,11681.557177,2
3,3,E,8518.072941,2
4,4,E,8722.482692,2
...,...,...,...,...
99995,49995,F,1.451000,2
99996,49996,F,1.454012,2
99997,49997,F,1.457035,2
99998,49998,F,1.456206,2


In [15]:
def gelman_plot(df: pd.DataFrame):

    assert isinstance(df, pd.DataFrame)

    plt = px.line(df,
                x = 'iteration',
                y = 'gelman',
                color = 'variable',
                title = 'Gelman tests for E and F as Gibbs sampler progresses',
                width = 1000,
                height =  500)
    
    plt.update_layout(yaxis_type = 'log')
    
    plt.write_image('gelman.png', scale = 8)


gelman_plot(gelman_df)

![Plot](gelman.png)
After around 40k iteration reach scale reduction factors (for both E and F) are close enough to 1 to assume stationarity of the distributions. I would suggest 40k burn-in time.

## 7

In [16]:
def count_autocorrelations(samples: np.ndarray, cols_of_interest: list,  k_max: int = 50) -> np.ndarray:

    assert isinstance(samples, np.ndarray)
    assert samples.ndim == 2
    assert all(isinstance(col, int) for col in cols_of_interest)
    assert all(0 <= col < samples.shape[1] for col in cols_of_interest)
    assert k_max < samples.shape[0]

    samples = samples[ : , cols_of_interest]

    means = np.mean(samples, axis=0)
    variances = np.var(samples, axis=0)
    autocorrelations = []

    for k in range(1, k_max + 1):
        covs = (samples[:-k, :] - means)*(samples[k:, :] - means)
        cor = np.mean(covs, axis = 0) / variances
        autocorrelations.append(cor)

    return np.array(autocorrelations)


autocorrelations = count_autocorrelations(samples2, [0, 1])
autocorrelations_df = stats_to_df(autocorrelations, 'autocorrelation', 2)
display(autocorrelations_df)

,iteration,variable,autocorrelation,sampler
0,0,E,0.556955,2
1,1,E,0.339864,2
2,2,E,0.214807,2
3,3,E,0.136357,2
4,4,E,0.093897,2
...,...,...,...,...
95,45,F,-0.001107,2
96,46,F,-0.003725,2
97,47,F,0.000732,2
98,48,F,-0.000041,2


In [19]:
def cor_plot(df: pd.DataFrame):

    assert isinstance(df, pd.DataFrame)

    plt = px.line(df,
                x = 'iteration',
                y = 'autocorrelation',
                color = 'variable',
                title = 'Autocorrelations between samples at lag = k',
                width = 1000,
                height = 500,
                labels = {'iteration':'k'})
    
    plt.write_image('correlations.png', scale = 8)


cor_plot(autocorrelations_df)

![Plot](correlations.png)

Autocorrelations oscilating over 0 at lag 15 and higher suggest that samples taken with such lags are independent. I would take the smallest of this lags (15) as a thinning-out parameter.

## 8

In [31]:
sampler_tuned = Sampler(model)
samples_tuned = sampler_tuned.sample(100, 40000, 15)
print('P(F = T|J = T, M = T) =', np.mean(samples_tuned[:, 1]))

P(F = T|J = T, M = T) = 0.16


The value of $P(F = T|J = T, M=T)$ estimated on the tuned samples after 40000 burn-in and 15 thin-out is similar to the basic one obtained without burn-in an thinning-out. It is hard to say, whether it is a better estimate without a systematic analysis. Thus, I compare those two approaches sampling many times with independent samplers (with basic and tuned parameters).

In [32]:
def compare_samplers(iters_compare: int, sampling_iters: int = 100, burn_in_iters: int = 0, thin_by: int = 1) -> np.ndarray:
    """Compares tuned sampling (with burn-in and thinning-out) to the basic sampling (without)."""

    assert isinstance(iters_compare, int)

    means = np.zeros((iters_compare, 2), dtype=float)
    variances = np.zeros((iters_compare, 2), dtype=float)

    for i in range(iters_compare):
        basic_sampler = Sampler(model)
        tuned_sampler = Sampler(model)

        basic_samples = basic_sampler.sample(sampling_iters)[:, 1]
        tuned_samples = tuned_sampler.sample(sampling_iters, burn_in_iters, thin_by)[:, 1]
        
        means[i][0] = np.mean(basic_samples)
        means[i][1] = np.mean(tuned_samples)

        variances[i][0] = np.var(basic_samples)
        variances[i][1] = np.var(tuned_samples)

    m = np.mean(means, axis = 0)
    v = np.mean(variances, axis = 0)

    return m, v


iters_compare = 100
sampling_iters = 500
burn_in_iters = 40000
thin_by = 15

means, variances = compare_samplers(iters_compare, sampling_iters=sampling_iters, burn_in_iters=burn_in_iters, thin_by=thin_by)

In [27]:
print(f'Mean and variance of P(F = T|J = T, M = T) for no burn-in and thinning-out is: {float(means[0].round(3)), float(variances[0].round(3))}')
print(f'Mean and variance of P(F = T|J = T, M = T) for {burn_in_iters=}, and {thin_by=} is: {float(means[1].round(3)), float(variances[1].round(3))}')

Mean and variance of P(F = T|J = T, M = T) for no burn-in and thinning-out is: (0.157, 0.131)
Mean and variance of P(F = T|J = T, M = T) for burn_in_iters=40000, and thin_by=15 is: (0.152, 0.128)


Systematic comparison of basic and tuned sampler shows, that the P(F = T|J = T, M = T) vary. One can not say, if predictions of on of them are more accurate without computing the probability analytically. But, because the variance between estimates is still high, I would use a bigger sample set to estimate P(F = T|J = T, M = T) reliably.

## 9

$P(E=T, F=T, J=T, M=T) = 0.038$  

$P(E=F, F=T, J=T, M=T) = 0.02375 $  

$P(E=T, F=F, J=T, M=T) = 0.324 $  

$P(E=F, F=F, J=T, M=T) = 0.0225 $

$P(F = T| J = T, M = T) = 
\frac{P(F=T, J=T, M=T)}{P(J=T, M=T)} = 
\frac{\sum_E P(E, F=T, J=T, M=T)}{\sum_{E,F} P(E, F, J=T, M=T)} = 
\frac{P(E=T, F=T, J=T, M=T) + P(E=F,F=T, J=T, M=T)}{P(E=T, F=T, J=T, M=T) + P(E=F,F=T, J=T, M=T) + P(E=T, F=F, J=T, M=T) + P(E=F, F=F, J=T, M=T)} = 
\frac{0.038 + 0.02375}{0.038 + 0.02375 + 0.324 + 0.0225} \approx 0.151$

The sampling estimates (mean = 0.152) obtained with tuned sampler are closer to the analytically calculated probability (0.151) that obtained with the basic sampler (mean = 0.160). That said, the chosen parameters helped in obtaining better estimate.